# Observer

It deals with One-to-Many relationships and utilizes events to let subscribed entities know about changes in an observable.
A subject and observer(many) exist in a one-to-many relationship. Here the observers do not have any access to data, so they are dependent on the subject to feed them with information.

## structure

In [163]:
# observable

id = 0

class Observable:
    def __init__(self):
        self.observers = []
    def notify(self, obj, attrib):
        for observer in self.observers:
            observer.attrib_changed(obj, attrib)
    def subscribe(self, observer):
        if observer not in self.observers:
            self.observers.append(observer)
    def unsubscribe(self, observer):
        if observer in self.observers:
            self.observers.remove(observer)
        

class Person(Observable):
    def __init__(self, name, age):
        super().__init__()
        global id 
        id += 1
        self._id = id
        self._name = name
        self._age = age
    
    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, name):
        if self._name == name:
            return
        else:
            self._name = name
            self.notify(self, "name")
            
    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, age):
        if self._age == age:
            return
        else:
            self._age = age
            self.notify(self, "age")
    @property
    def id(self):
        return self._id

In [168]:
# observer

class Observer:
    @abstractmethod
    def attrib_changed(self, obj, attrib):
        pass

class ConsolePersonObserver(Observer):
    def attrib_changed(self, obj, attrib):
        if hasattr(obj, attrib):
            if not ismethod(getattr(obj, attrib)):
                print(f"Person:{obj.id} has changed {attrib} to {getattr(obj, attrib)}")
            else:
                print(f"person:{obj.id} has changed {attrib} to {eval("o."+attrib+"()", globals(), {"o":obj})}")

In [165]:
# connect observers and observables

p = Person("John", 45)
ob = ConsolePersonObserver()
p.subscribe(ob)
p.name = "lisa"
p.age = 34

Person:1 has changed name to lisa
Person:1 has changed age to 34


## dependency

In cases where there are property dependencies, we have to track manually the dependencies.

In [166]:
# class with dependencies

class CivicPerson(Person):
    def __init__(self, name, age, children):
        super().__init__(name, age)
        self._children = children
        self.dependencies = {}
        self.dependencies["age"] = ["can_vote", "can_drive"]
        self.dependencies["children"] = ["have_welfare"]

    def can_vote(self):
        return self.age > 18

    def can_drive(self):
        return self.age > 16

    def have_welfare(self):
        return self.children > 0
        
    @property
    def age(self):
        return self._age
        
    @age.setter
    def age(self, age):
        if self._age == age:
            return
        else:
            old_attr = [eval("self."+attr)() for attr in self.dependencies["age"]]
            self._age = age
            self.notify(self, "age")
            for i, attr in enumerate(self.dependencies["age"]):
                if eval("self."+attr)() != old_attr[i]:
                    self.notify(self, attr)

    @property
    def children(self):
        return self._children

    @children.setter
    def children(self, children):
        if self._children == children:
            return
        else:
            old_attr = [eval("self."+attr)() for attr in self.dependencies["children"]]
            self._children = children
            self.notify(self, "children")
            for i, attr in enumerate(self.dependencies["children"]):
                if eval("self."+attr)() != old_attr[i]:
                    self.notify(self, attr)
            

In [169]:
p = CivicPerson("John", 10, 0)
ob = ConsolePersonObserver()
p.subscribe(ob)
p.name = "lisa"
p.age = 34
p.children = 1

Person:3 has changed name to lisa
Person:3 has changed age to 34
person:3 has changed can_vote to True
person:3 has changed can_drive to True
Person:3 has changed children to 1
person:3 has changed have_welfare to True


## thread safety

The list ifself is thread safe, which allows simultaneous read and write, but not safe for simultaneous add and remove. I am not clear on this, but there is nothing wrong to make it safer.
If not safe, concurrent subscribe and unsubscribe can cause problem for observers list in the class Observable.
In other language such c++, vector is definitly not thread safe.

In [ ]:
# add mutex 

from threading import Lock

mutex = Lock()

def processData(data):
    with mutex:
        print('Do some stuff')

class Observable:
    def __init__(self):
        self.observers = []
    def notify(self, obj, attrib):
        with mutex:
            for observer in self.observers:
                observer.attrib_changed(obj, attrib)
    def subscribe(self, observer):
        with mutex:
            if observer not in self.observers:
                self.observers.append(observer)
    def unsubscribe(self, observer):
        with mutex:
            if observer in self.observers:
                self.observers.remove(observer)

## refs

https://www.learnerlandmark.com/post/observer-vs-pub-sub-pattern